In [17]:
import pandas as pd

read_path = "../data/comments_clean.h5"
df = pd.read_hdf(read_path, key='df')
df.sample(3)

,id,createdAt,userId,userCreatedAt,classYear,universityId,country,universityName,parentId,questionId,comment,review,likes,dislikes
23008,28662,2023-06-13 12:26:08.651916+00:00,34341,2023-05-13 10:22:41.790573+00:00,Year 4,2632,United Kingdom,University of Central Lancashire,26589.0,3780,Thanks :),False,0,0
8617,12135,2022-06-14 22:02:31.832509+00:00,18741,2022-03-15 20:23:37.606045+00:00,Year 3,2670,United Kingdom,University of Southampton,8565.0,5031,I swear they keep changing it,True,0,0
341,627,2021-04-15 09:44:32.478000+00:00,1195,2019-03-18 18:41:10.891000+00:00,Year 4,2620,United Kingdom,University College London (UCL),390.0,3077,No it's not,True,2,0


In [18]:
tone_labels = ['negative', 'neutral', 'positive']
emotion_labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
theme_labels = ['clinical update', 'community', 'question', 'education', 'advocating', 'dissuading', 'other']

In [26]:
import json
import os


df['text'] = df['comment']
df['meta'] = ""

df['meta'] = (
  df['id'].apply(lambda x: f'{{"annotated": true, "id": "{x}", ') +
  df['createdAt'].apply(lambda x: f'"createdAt": "{x.strftime("%d/%m/%Y")}", ') +
  df['likes'].apply(lambda x: f'"likes": {x}, ') +
  df['dislikes'].apply(lambda x: f'"dislikes": {x}}}')
)

filename = f'../datasets/comments.jsonl'
df.loc[:, ['text', 'meta']].to_json(filename, orient='records', lines=True)
tmp_filename = '../datasets/tmp.jsonl'

with open(filename, 'r') as f:
  with open(tmp_filename, 'w') as f2:
    for line in f:
      row = json.loads(line)
      try:
        row['meta'] = json.loads(row['meta'])
      except:
        print(row['meta'])
        raise
      f2.write(json.dumps(row, ensure_ascii=False) + "\n")

os.remove(filename)
os.rename(tmp_filename, filename)